In [ ]:
import chess
import chess.engine
from typing import List, Tuple

def score_to_cp(score, mate_score=100000):
    """Convert a python-chess Score to centipawn-like integer (mate -> large positive/negative)."""
    if score.is_mate():
        # positive means mate for the side the score is measured for
        mate_in = score.mate()
        return mate_score if mate_in > 0 else -mate_score
    else:
        return score.score()

def worst_moves(
    fen: str,
    engine_path: str = "stockfish",
    depth: int = 12,
    reply_depth: int = 6,
    top_n: int = 3,
    use_reply_eval: bool = True,
) -> List[Tuple[str, int, int, bool]]:
    """
    Returns top_n worst moves as tuples: (uci_move, eval_after_move_cp, eval_after_reply_cp, allows_opponent_mate_bool)
    evals are from perspective of the side to move at the starting position (lower = worse).
    """
    board = chess.Board(fen)
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        # baseline eval (from side-to-move perspective)
        base_info = engine.analyse(board, limit=chess.engine.Limit(depth=depth))
        base_score = score_to_cp(base_info["score"].pov(board.turn))

        results = []
        legal_moves = list(board.legal_moves)

        for mv in legal_moves:
            board.push(mv)
            # Evaluate position right after our move
            info_after = engine.analyse(board, limit=chess.engine.Limit(depth=depth))
            eval_after_move = score_to_cp(info_after["score"].pov(board.turn))  # pov board.turn since it's still that color's POV
            # But we want score from original side-to-move perspective:
            # If side to move was White and turned to Black, pov(board.turn) returns from Black's POV.
            # Simpler approach: get score from original perspective:
            eval_after_move_from_original = score_to_cp(info_after["score"].pov(not board.turn))

            allows_opponent_mate = False
            eval_after_reply_from_original = None

            if use_reply_eval:
                # Ask engine to play best reply for opponent (opponent is now board.turn)
                reply = engine.play(board, limit=chess.engine.Limit(depth=reply_depth))
                if reply.move is not None:
                    board.push(reply.move)
                    info_after_reply = engine.analyse(board, limit=chess.engine.Limit(depth=depth))
                    eval_after_reply_from_original = score_to_cp(info_after_reply["score"].pov(not board.turn))
                    # detect if opponent delivered mate
                    if info_after_reply["score"].is_mate():
                        # if mate is negative from original side, opponent mates us
                        m = info_after_reply["score"].mate()
                        # if mate positive means mate for side whose pov we used; we used not board.turn = original side?
                        # simpler: if eval is extremely negative, treat as mate for opponent
                        allows_opponent_mate = eval_after_reply_from_original <= -90000
                    board.pop()
                else:
                    # no reply move? (shouldn't happen)
                    eval_after_reply_from_original = eval_after_move_from_original

            results.append((
                mv.uci(),
                eval_after_move_from_original,
                eval_after_reply_from_original if eval_after_reply_from_original is not None else eval_after_move_from_original,
                allows_opponent_mate
            ))
            board.pop()

        # Sort by worst (lowest eval) after reply if using reply eval else after move
        key_index = 2 if use_reply_eval else 1
        results.sort(key=lambda x: x[key_index])  # lower is worse for us

        return results[:top_n]

if __name__ == "__main__":
    # Example: a position where White to move
    fen = "r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 2 4"
    engine_path = "/usr/bin/stockfish"  # change to your Stockfish path
    bad = worst_moves(fen, engine_path=engine_path, depth=10, reply_depth=6, top_n=5, use_reply_eval=True)
    for uci, eval_after_move, eval_after_reply, allows_mate in bad:
        print(f"{uci} | eval_after_move={eval_after_move} | eval_after_reply={eval_after_reply} | allows_mate={allows_mate}")